**Mount Drive**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


 **Install diffrent Packages**


In [3]:
!apt-get -qq install -y libarchive-dev && pip install -U libarchive
import libarchive
!pip install d2l==0.17.0
!pip install -U mxnet-cu101==1.7.0

Selecting previously unselected package libarchive-dev:amd64.
(Reading database ... 155013 files and directories currently installed.)
Preparing to unpack .../libarchive-dev_3.2.2-3.1ubuntu0.7_amd64.deb ...
Unpacking libarchive-dev:amd64 (3.2.2-3.1ubuntu0.7) ...
Setting up libarchive-dev:amd64 (3.2.2-3.1ubuntu0.7) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
     |████████████████████████████████| 154 kB 7.5 MB/s 
  Created wheel for libarchive: filename=libarchive-0.4.7-py3-none-any.whl size=31647 sha256=42d16812377051dcd37bb2aacf629a865b74f63396b42a123aa4e54ddde637e1
  Stored in directory: /root/.cache/pip/wheels/63/b1/c6/b3da79bec2012175bd43603eed98ef8548ac1733b77c1d4330
Successfully built libarchive
     |████████████████████████████████| 83 kB 1.2 MB/s 
     |███████████████████████████████▌| 834.1 MB 1.2 MB/s eta 0:00:10tcmalloc: large alloc 1147494400 bytes == 0x55720d2a8000 @  0x7f456de71615 0x5571d31794cc 0x5571d325947a 0x5571d317c2ed 0x5571d326de1d 0x5571d31efe99

**Import Required Models And Libararies**

In [4]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
import random
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


**Import Dataset File**

In [5]:
#Load and Read Intents.json file from My Drive
intents_file = open('/content/drive/MyDrive/Colab Notebooks/intents.json').read()
intents = json.loads(intents_file)

**Tokenization, Corpus And Classes List**

In [6]:
#Declare Some Lists
words=[]
classes = []
documents = []
ignore_letters = ['!', '?', ',', '.']
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #Tokenize Each Word
        word = nltk.word_tokenize(pattern)
        words.extend(word)        
        #Add Documents in the Corpus
        documents.append((word, intent['tag']))
        #Add to our Classes List
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
print(documents)

[(['Hi'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['What', 'hours', 'are', 'you', 'open', '?'], 'hours'), (['What', 'are', 'your', 'hours', '?'], 'hours'), (['When', 'are', 'you', 'open', '?'], 'hours'), (['When', 'is', 'the', 'time', 'to', 'contact', '?'], 'hours'), (['At', 'what', 'time', 'do', 'you', 'provide', 'services', '?'], 'hours'), (['What', 'is', 'your', 'location', '?'], 'location'), (['Where', 'are', 'you', 'located', '?'], 'location'), (['What', 'is', 'your', 'address', '?'], 'location'), (['Where', 'is', 'your', 'company', 'situated', '?'], 'location'), (['How', 'can', 'we', 'contact', 'you', '?'], 'location'), (['How', 'can', 'I', 'contact', 'you', '?'], 'location'), (['Do', 'you', '

**Lemmaztize, Duplicates, Sorting, Vocabulary,Combinations**

In [7]:
#Lemmaztize and Lower each Word and Remove Duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_letters]
words = sorted(list(set(words)))
#Sort Classes
classes = sorted(list(set(classes)))
# Intents, Documents
print (len(documents), "documents")
# Intents, Classes
print (len(classes), "classes", classes)
# Words = All words, Vocabulary
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

106 documents
37 classes [' file infector virus', 'ARP', 'Benefits of cybersecurity', 'Botnet', 'Cognitive Cybersecurity', 'Cryptography', 'Decryption', 'Encryption different from Hashing', 'Features of a Cybersecurity', 'Firewall', 'Forward Secrecy', 'Grey hat hackers', 'Hashing', 'Penetration Testing', 'Secure Sockets Layers', 'Transport Layer Security', 'VPN', 'Vulnerability Assessment', 'White hat hackers', 'black hat hackers', 'boot sector virus', 'computer virus', 'cybersecurity', 'direct action virus', 'encryption', 'goodbye', 'greeting', 'hours', 'location', 'multipartite virus', 'payments', 'resident virus', 'services', 'spacefiller virus', 'thanks', 'traceroute', 'training']
124 unique lemmatized words ["'s", 'a', 'accept', 'action', 'address', 'an', 'and', 'anyone', 'are', 'arp', 'assessment', 'at', 'benefit', 'black', 'bonet', 'boot', 'botnet', 'by', 'bye', 'can', 'card', 'cash', 'cognitive', 'company', 'computer', 'conduct', 'contact', 'credit', 'cryptography', 'cyber', 'c

**Training Data,Empty Array for output, Numpy Array and Testing**

In [8]:
# Training data
training = []
# Empty array for the output
output_empty = [0] * len(classes)
# Training set, bag of words for every sentence
for doc in documents:
    # Bag of words
    bag = []
    # list of tokenized words for the pattern
    word_patterns = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    # Create the bag of words array with 1, if word is found in current pattern
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
# shuffling and Numpy array
random.shuffle(training)
training = np.array(training)
# Training and Testing
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data is created")

Training data is created


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


**Deep Learning Model, Training and Seving**

In [9]:
#Deep Neural Networds Model
#Relu and Softmax Activation Function
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
# Compiling model. SGD with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#Training with 200 epoches and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('/content/drive/MyDrive/Colab Notebooks/chatbot_model.h5', hist)
print("model is created")

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/200
22/22 [==============================] - 1s 3ms/step - loss: 3.6524 - accuracy: 0.0283
Epoch 2/200
22/22 [==============================] - 0s 1ms/step - loss: 3.5470 - accuracy: 0.0472
Epoch 3/200
22/22 [==============================] - 0s 1ms/step - loss: 3.5084 - accuracy: 0.0660
Epoch 4/200
22/22 [==============================] - 0s 1ms/step - loss: 3.4957 - accuracy: 0.0943
Epoch 5/200
22/22 [==============================] - 0s 1ms/step - loss: 3.3650 - accuracy: 0.0849
Epoch 6/200
22/22 [==============================] - 0s 2ms/step - loss: 3.2994 - accuracy: 0.1415
Epoch 7/200
22/22 [==============================] - 0s 1ms/step - loss: 3.2464 - accuracy: 0.1604
Epoch 8/200
22/22 [==============================] - 0s 1ms/step - loss: 3.0048 - accuracy: 0.1887
Epoch 9/200
22/22 [==============================] - 0s 2ms/step - loss: 2.9919 - accuracy: 0.1887
Epoch 10/200
22/22 [==============================] - 0s 2ms/step - loss: 2.9036 - accuracy: 0.2358
Epoch 11/